# Langchain - Memory

Large Language Models (LLMs) do not "remember" any of your "previous" conversations - each interaction is distinct from the other. This is an issue, especially when you are building chatbots, where you want to have a conversation where LLM must "know" previous conversations in the thread.

Langchain provides "Memory" which helps LLM "remember" previous interactions and feed that into the model, so you can have the conversational flow as you chat with them.

In this workbook we'll cover
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

In [2]:
# load my LLM keys from .env files
import os
import warnings

warnings.filterwarnings("ignore")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [16]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(
    temperature=0.0,
    model="gpt-3.5-turbo",  # enough for this tutorial
    max_tokens=2048,
    max_retries=2,
    verbose=False,
)
print(chat_model)

client=<openai.resources.chat.completions.Completions object at 0x0000022A82EBEEA0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000022A82EC1B20> root_client=<openai.OpenAI object at 0x0000022A81B94770> root_async_client=<openai.AsyncOpenAI object at 0x0000022A82EBE480> temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') max_retries=2 max_tokens=2048


In [24]:
def get_response(chat_model, message, config=None):
    messages = [
        (
            "system",
            "You are a helpful assistant that responds to the user in a calm and respectful tone",
        ),
        ("human", message),
    ]
    if config is not None:
        response = chat_model.invoke(messages, config=config)
    else:
        response = chat_model.invoke(messages)
    return response

In [18]:
# now let's start chatting
response = get_response(
    chat_model, "Hi! My name is Manish and I am new to Generative AI"
)
print(response.content)

Hello Manish! Welcome to the world of Generative AI. I'm here to help you with any questions you may have. What would you like to know or learn about Generative AI?


In [20]:
response = get_response(chat_model, "What was my name?")
print(response.content)

I'm sorry, but I don't have access to personal information like your name. How can I assist you today?


In [21]:
# to memory enable the LLM
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

# this is my message & response store
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


chat_model_with_memory = RunnableWithMessageHistory(chat_model, get_session_history)

In [22]:
from datetime import datetime

now_str = datetime.now().strftime("%Y%m%d_%H%M%S")
session_id = f"mjb4genai{now_str}"

config = {"configurable": {"session_id": session_id}}

In [25]:
response = get_response(
    chat_model_with_memory,
    "Hi! My name is Manish and I am new to Generative AI",
    config=config,
)
print(response.content)

Hello Manish! Welcome to the world of Generative AI. I'm here to help you with any questions you may have. What would you like to know or learn about Generative AI?


In [26]:
print(store)

{'mjb4genai20250113_115748': InMemoryChatMessageHistory(messages=[SystemMessage(content='You are a helpful assistant that responds to the user in a calm and respectful tone', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi! My name is Manish and I am new to Generative AI', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Manish! Welcome to the world of Generative AI. I'm here to help you with any questions you may have. What would you like to know or learn about Generative AI?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 42, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-77a73ef4-e0f6

In [27]:
response = get_response(
    chat_model_with_memory,
    "What is an AI Agent?",
    config=config,
)
print(response.content)

An AI agent is a software program that acts on behalf of a user or another program to perform specific tasks or make decisions. AI agents can be designed to operate autonomously or with human guidance, depending on the application. They are commonly used in various fields such as robotics, virtual assistants, and autonomous vehicles to assist users in completing tasks efficiently. If you have any more questions or need further clarification, feel free to ask!


In [28]:
print(store)

{'mjb4genai20250113_115748': InMemoryChatMessageHistory(messages=[SystemMessage(content='You are a helpful assistant that responds to the user in a calm and respectful tone', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi! My name is Manish and I am new to Generative AI', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Manish! Welcome to the world of Generative AI. I'm here to help you with any questions you may have. What would you like to know or learn about Generative AI?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 42, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-77a73ef4-e0f6

In [30]:
response = get_response(
    chat_model_with_memory,
    "What was my name?",
    config=config,
)
print(response.content)

Your name is Manish. How can I assist you further, Manish?
